```
huggingface-cli login
```
give the read token when prompted!
then for windows:
```
Enable Developer Mode on Windows (recommended long-term solution):
```

updated jypyternotebook: 
```
pip install ipywidgets
```

packages
```

!pip install datasets
!pip install torch transformers llama_index tqdm 
!pip install einops
!pip install qdrant_client
```

In [5]:
# from datasets import load_dataset

# # Step 1: Load the SQuAD dataset
# dataset = load_dataset("squad")

# # Step 2: Extract unique contexts{set} from the dataset
# data = [item["context"] for item in dataset["train"]]
# texts = list(set(data))

c:\Users\Bidhi\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Bidhi\.cache\huggingface\hub\datasets--squad. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Generating validation split: 100%|██████████| 10570/10570 [00:00<00:00, 397783.74 examples/s]


In [20]:
# texts[0]
# import sys
# size = sum(sys.getsizeof(text) for text in texts) + sys.getsizeof(texts)  # Sum elements + list overhead
# size_in_mb = size / (1024*1024)

# print(f"Size of text list is {size_in_mb:.3f} MB")

Size of text list is 17.656 MB


In [27]:

def batch_iterate(lst, batch_size):
    for i in range(0, len(lst), batch_size):
        yield lst[i : i + batch_size]

In [24]:
# from llama_index.embeddings.huggingface import HuggingFaceEmbedding
# # from tqdm import tqdm
# #for jupyternotebook
# from tqdm.notebook import tqdm


# class EmbedData:

#     def __init__(self, 
#                  embed_model_name="nomic-ai/nomic-embed-text-v1.5",
#                  batch_size=32):
        
#         self.embed_model_name = embed_model_name
#         self.embed_model = self._load_embed_model()
#         self.batch_size = batch_size
#         self.embeddings = []

#     def _load_embed_model(self):
#         embed_model = HuggingFaceEmbedding(model_name=self.embed_model_name,
#                                            trust_remote_code=True,
#                                            cache_folder='./hf_cache')
#         return embed_model

    
#     def generate_embedding(self, context):
#         return self.embed_model.get_text_embedding_batch(context)
    
#     def embed(self, contexts):
#         self.contexts = contexts
        
#         total_batches = (len(contexts) + self.batch_size - 1) // self.batch_size

#         with tqdm(total=total_batches, desc="Embedding data", unit="batch") as pbar:
#             for batch_context in batch_iterate(contexts, self.batch_size):
#                 batch_embeddings = self.generate_embedding(batch_context)
#                 self.embeddings.extend(batch_embeddings)
#                 pbar.update(1)  


# batch_size = 32

# embeddata = EmbedData(batch_size=batch_size)

# embeddata.embed(texts)

In [30]:
import pickle
with open("embeddings_data.pkl", "rb") as d:
    embeddata = pickle.load(d)

embeddata["contexts"][0]


default_segment_number=5: This means the collection will start with 5 segments, which can help balance performance and memory efficiency. More segments allow parallelism but might slightly increase query latency due to searching multiple segments.

The indexing_threshold parameter controls when Qdrant should enable HNSW indexing for faster queries.
  If indexing_threshold is set to 0 → Qdrant immediately creates an HNSW index, no matter how few vectors exist.

  If indexing_threshold is set to a higher value (e.g., 10000) → Qdrant will wait until there are at least 10,000 vectors before enabling HNSW indexing.

In [128]:
from qdrant_client import QdrantClient
from qdrant_client import models
from tqdm import tqdm 

def batch_iterate(lst, batch_size):
    for i in range(0, len(lst), batch_size):
        yield lst[i : i + batch_size]

class QdrantVDB:

    def __init__(self, collection_name, vector_dim=768, batch_size=512):
        self.collection_name = collection_name
        self.batch_size = batch_size
        self.vector_dim = vector_dim

    def define_client(self):
        self.client = QdrantClient(url="http://localhost:6333",
                                   prefer_grpc=True)
        #gRPC is a communication mechanism that offers faster interactions compared to HTTP.

    def create_collection(self):
            
            if not self.client.collection_exists(collection_name=self.collection_name):

                self.client.create_collection(collection_name=self.collection_name,
                                            
                                            vectors_config=models.VectorParams(
                                                                size=self.vector_dim,
                                                                distance=models.Distance.DOT, #dot product similarity 
                                                                on_disk=True), # whether to store the vectors on disk instead of memory, which optimizes memory usage for larger datasets.
                                            
                                            optimizers_config=models.OptimizersConfigDiff(
                                                                                default_segment_number=5,
                                                                                indexing_threshold=0)
                )


                
    def ingest_data(self, embeddata):
    
        for batch_context, batch_embeddings in tqdm(zip(batch_iterate(embeddata["contexts"], self.batch_size), 
                                                        batch_iterate(embeddata["embeddings"], self.batch_size)), 
                                                    total=len(embeddata["contexts"])//self.batch_size, 
                                                    desc="Ingesting in batches"):
        
            self.client.upload_collection(collection_name=self.collection_name,
                                        vectors=batch_embeddings,
                                        payload=[{"context": context} for context in batch_context])

            self.client.update_collection(collection_name=self.collection_name,
                                    optimizer_config=models.OptimizersConfigDiff(indexing_threshold=20000)
                                    )
            
    def create_collection_fast(self):
        
        if not self.client.collection_exists(collection_name=self.collection_name):

            self.client.create_collection(collection_name=self.collection_name,
                                          
                                          vectors_config=models.VectorParams(
                                                              size=self.vector_dim,
                                                              distance=models.Distance.DOT,
                                                              on_disk=True),
                                          
                                          optimizers_config=models.OptimizersConfigDiff(
                                                                            default_segment_number=5,
                                                                            indexing_threshold=0),
                                          
                                          quantization_config=models.BinaryQuantization(
                                                        binary=models.BinaryQuantizationConfig(always_ram=True)),
                                         )

database = QdrantVDB("squad_collection")
database.define_client()
database.create_collection()
database.ingest_data(embeddata)

Ingesting in batches: 37it [00:15,  2.38it/s]                        


In [116]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from qdrant_client import models

embed_model_name="nomic-ai/nomic-embed-text-v1.5"

embed_model = HuggingFaceEmbedding(
            model_name=embed_model_name,
            trust_remote_code=True,
            cache_folder="./hf_cache"
        )

<All keys matched successfully>


In [130]:
import time

class Retriever:
    def __init__(self, vector_db, embed_model ):
        self.vector_db = vector_db
        self.embed_model = embed_model

    def search(self, query):
        """Search for similar embeddings in the Qdrant database."""

        query_embedding = embed_model.get_text_embedding(query)
            
        # Start the timer
        start_time = time.time()
        
        result = self.vector_db.client.search(
            collection_name=self.vector_db.collection_name,
            
            query_vector=query_embedding,
            
            search_params=models.SearchParams(
                quantization=models.QuantizationSearchParams(
                    ignore= True,  # this need to be false to use it with fast search aka binary quantization
                    rescore=True,
                    oversampling=2.0,
                )
            ),
            
            timeout=1000,
        )
        
        # End the timer
        end_time = time.time()
        elapsed_time = end_time - start_time

        print(f"Execution time for the search: {elapsed_time:.4f} seconds")

        return result

In [131]:
results = Retriever(database, embed_model).search("Sample query")
print(results)

C:\Users\Bidhi\AppData\Local\Temp\ipykernel_564\2666107564.py:16: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  result = self.vector_db.client.search(


Execution time for the search: 0.3095 seconds
[ScoredPoint(id='3c050027-3103-4ba0-9e8a-50ef612c08e4', version=906, score=0.5884716510772705, payload={'context': 'Static analysis techniques for software verification can be applied also in the scenario of query languages. In particular, the *Abstract interpretation framework has been extended to the field of query languages for relational databases as a way to support sound approximation techniques. The semantics of query languages can be tuned according to suitable abstractions of the concrete domain of data. The abstraction of relational database system has many interesting applications, in particular, for security purposes, such as fine grained access control, watermarking, etc.'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id='4028fe72-e637-4d95-8dbe-3080d049a1ee', version=610, score=0.5884716510772705, payload={'context': 'Static analysis techniques for software verification can be applied also in the scenario of qu

```
pip install llama-index qdrant_client torch transformers

pip install llama-index-embeddings-huggingface

pip install llama-index-llms-ollama

pip install llama-index-vector-stores-qdrant


In [132]:
from llama_index.llms.ollama import Ollama

class RAG:

    def __init__(self,
                 retriever,
                 llm_name="llama3.2:1b"):
        
        self.llm_name = llm_name
        self.llm = self._setup_llm()
        self.retriever = retriever
        self.qa_prompt_tmpl_str = """Context information is below.
                                     ---------------------
                                     {context}
                                     ---------------------
                                     
                                     ---------------------
                                     Query: {query}
                                     ---------------------
                                     
                                     Answer: """
        
    def _setup_llm(self):
            return Ollama(model=self.llm_name)
        
    def generate_context(self, query):
    
            result = self.retriever.search(query)
            context = [dict(data) for data in result]
            combined_prompt = []

            for entry in context:
                context = entry["payload"]["context"]

                combined_prompt.append(context)

            return "\n\n---\n\n".join(combined_prompt)
        
    def query(self, query):
            context = self.generate_context(query=query)
            
            prompt = self.qa_prompt_tmpl_str.format(context=context, query=query)
            
            response = self.llm.complete(prompt)
            
            return dict(response)['text']
        




In [133]:
retriever = Retriever(database, embeddata)

rag = RAG(retriever)

In [92]:
Retriever(database, embeddata).search("Sample query")[0]
###
embeddata["contexts"][11095]

Execution time for the search: 0.0079 seconds


C:\Users\Bidhi\AppData\Local\Temp\ipykernel_564\3344487341.py:16: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  result = self.vector_db.client.search(


"Airports may also contain premium and VIP services. The premium and VIP services may include express check-in and dedicated check-in counters. These services are usually reserved for First and Business class passengers, premium frequent flyers, and members of the airline's clubs. Premium services may sometimes be open to passengers who are members of a different airline's frequent flyer program. This can sometimes be part of a reciprocal deal, as when multiple airlines are part of the same alliance, or as a ploy to attract premium customers away from rival airlines."

In [136]:
from IPython.display import Markdown, display

query = """The premium and VIP services in Airports
           are reserved for which type of passengers?"""

answer = rag.query(query)

display(Markdown(str(answer)))

C:\Users\Bidhi\AppData\Local\Temp\ipykernel_564\2666107564.py:16: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  result = self.vector_db.client.search(


Execution time for the search: 0.0038 seconds


The text does not specifically state the type of passengers that premium and VIP services are reserved for. However, it can be inferred that they are typically intended for First and Business class passengers, as well as members of airline's clubs.